Prototyp v0.1  zur Evaluierung der Analyse von  Mediationstranskipten via LLMs  

In [ ]:
#all Imports, Configs and Regex 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
from collections import defaultdict
import openai
import time 
import re
import inspect
import pandas as pd


#load transkripts

# Funktion zum Laden des Konversationstexts aus einer Datei
def load_conversation_from_file(file_path):
    with open(file_path, 'r') as file:
        conversation_text = file.read()
    return conversation_text


file_path = 'med.txt'
# Konversationstext aus der Datei laden
conversation_text_gen = load_conversation_from_file(file_path)

file_path = 'realBsp.txt'
conversation_text_real = load_conversation_from_file(file_path)

# Split the conversation into individual lines
lines_gen = [line_gen.strip() for line_gen in conversation_text_gen.split("\n")]
lines_real = [line_real.strip() for line_real in conversation_text_real.split("\n")]

# Remove empty strings
lines_gen = list(filter(None, lines_gen))
lines_real = list(filter(None, lines_real))




#LLM Configs


openai_config_3 = {
    'model_name': 'gpt-3.5-turbo',
    'key': 'xx'
}
openai_config_4 = {
    'model_name': 'gpt-4',
    'key': 'xx'
}
openai_config_13B = {
    'model_name': 'llama2 13B',
    'base': 'http://k3:4321/v1', #works on workstation k3
    'key': 'cc' 
}
openai_config_70B = {
    'model_name': 'llama2 70B',
    'base': 'http://k3:1234/v1', #works on workstation k3
     'key': 'cc'
}
openai_config_pol  = {
    'model_name': 'polarity',
    'pol':1 
 
}



Statistische Analyse der Redeanteile

In [ ]:
import re
import matplotlib.pyplot as plt

# Define a dictionary to map speakers to colors
speaker_colors = {
        "Sprecher I": "orange",
        "Sprecher B1": "olive",
        "Sprecher B2": "cornflowerblue",
        "Sprecher M": "orange",
        "Sprecher B": "olive",
        "Sprecher A": "cornflowerblue",
        # Add more speaker-color mappings here
    }

def extract_speaker_stats(conversation_text, title):

    # Extract speakers and count the total number of words for each speaker
    sprecher_beitraege = re.split(r'(?=[A-Z]+\d*:)', conversation_text)
    gesamt_woerter_pro_sprecher = {}

    for beitrag in sprecher_beitraege:
        sprecher_match = re.match(r'([A-Z]+\d*):', beitrag)
        if sprecher_match:
            sprecher = f"Sprecher {sprecher_match.group(1)}"
            gesamt_woerter_pro_sprecher[sprecher] = len(re.findall(r'\w+', beitrag))

    # Calculate the percentage of speech time
    gesamt_woerter = sum(gesamt_woerter_pro_sprecher.values())
    prozentuale_redeanteile = {sprecher: (woerter / gesamt_woerter) * 100 for sprecher, woerter in gesamt_woerter_pro_sprecher.items()}

    # Extract the speaker names from the dictionary keys
    speakers = prozentuale_redeanteile.keys()

    # Get the colors for each speaker based on the speaker-color dictionary
    colors = [speaker_colors.get(speaker, "gray") for speaker in speakers]

    # Plot the pie chart with the assigned colors
    plt.pie(prozentuale_redeanteile.values(), labels=speakers, colors=colors, autopct='%1.1f%%')
    plt.title(title)
    plt.show()

# Generate pie chart for 'generierte Mediation'
extract_speaker_stats(conversation_text_gen, 'Prozentualer Redeanteil nach Sprechern für generierte Mediation')

# Generate pie chart for 'realen Mediation'
extract_speaker_stats(conversation_text_real, 'Prozentualer Redeanteil nach Sprechern für realen Mediation')

Sequenzielle Sentiment Analysen

In [ ]:
import time
from textblob import TextBlob
import pandas as pd

def extract_float(text):
    numbers = re.findall(r'-?\d+\.\d+', text)
    return float(numbers[0]) if numbers else 0

#Function for send Prompt to LLM 
def ask_ki(prompt,model_name):
    # Send the user prompt to ChatGPT using the correct endpoint for chat completions
    response = openai.ChatCompletion.create(
        model=model_name,
        messages=[{"role": "system", "content": "You are a helpful assistant"},
                  {"role": "user", "content": prompt}],
        max_tokens=100)#,
        #temperature = 0)#,
        #top_k=3)    # Adjust as needed
    return response


def analyze_sentiments(openai_config, lines,file_name, table_Sent):
    # Startzeit für die Messung der benötigten Zeit
    start_time = time.time()
 
    # Recognize speakers based on lines starting with a single uppercase letter followed by a colon
    

    # Initialize dictionaries to store sentiment and speaker parts
    sentiment_per_speaker = defaultdict(list)
    speaker_lengths = {}
    sentiments_over_time = defaultdict(list)
    if 'model_name' in openai_config:
        model_name = openai_config['model_name']
    if 'key' in openai_config:
        openai.api_key = openai_config['key']
    
    if 'base' in openai_config:
        openai.api_base = openai_config['base']

    
    current_speaker = None
    # Analyze sentiment for each line using the OpenAI API
    for idx, line in enumerate(lines, start=1):
    # Check if the line starts with a single uppercase letter followed by a colon
        
                
       # if len(line) > 2 and line[1] == ':' and line[0].isalpha() and line[0].isupper():
          # current_speaker = line.split(':')[0].strip()

             
        if len(line) > 3:
            match = re.match(r'^([A-Z\d]+):', line)
            if match:
                current_speaker = match.group(1)

        #line = re.sub(r'\(\d+\.\d+\)$', '', line)

        if 'pol' in openai_config:
               sentiment_value= TextBlob(line).sentiment.polarity 
        else:
            # User prompt for ChatGPT, including the sentiment analysis request
            prompt = f"{current_speaker}: {line}\n Rate the sentiment on a scale from -1 to 1, Give only the Float Number as result: "
            response = ask_ki(prompt,model_name)
    
       
             # Extract sentiment information from the response
            sentiment_label = response['choices'][0]['message']['content'].strip()
            sentiment_value= extract_float(sentiment_label)
    
        #Datentabelle
        #print(f"Sprecher: {current_speaker} Text: {line} Sentiment: {sentiment_value} ")  

        
        # Überprüfen, ob die Zeile existiert (zum Beispiel anhand des Namens)
        if not any(table_Sent['idx'] == idx):
         # Füge eine neue Zeile für  hinzu, falls sie nicht existiert
                table_Sent = table_Sent.append({'idx': idx, 'Sprecher': current_speaker, 'Text': line}, ignore_index=True)

        # Wert für eine bestimmte Zelle setzen, auch wenn die Zeile gerade erst hinzugefügt wurde
        table_Sent.at[table_Sent[table_Sent['idx'] == idx].index[0], openai_config['model_name']] = sentiment_value

        #print(table_Sent) 

        # Store sentiment information
        sentiment_per_speaker[current_speaker].append(sentiment_value)
        
        # Store speaker part lengths
        speaker_lengths[current_speaker] = len(line)
    
        # Store sentiments over time
        sentiments_over_time[current_speaker].append({"sentiment": sentiment_value, "time": idx})



        # Create a timeline chart for each speaker
    for speaker, time_sentiments in sentiments_over_time.items():
        times = [entry["time"] for entry in time_sentiments]
        sentiments = [entry["sentiment"] for entry in time_sentiments]
        plt.plot(times, sentiments, label=speaker)

    # Berechne die verbrauchten Tokens
    if 'pol' in openai_config:
        consumed_tokens = 0
    else:    
        consumed_tokens = response['usage']['total_tokens']

    #  Endzeit für die Messung der benötigten Zeit
    end_time = time.time()


    # Berechne die benötigte Zeit
    elapsed_time = end_time - start_time

    
    print(f'Consumed Tokens: {consumed_tokens} - Elapsed Time: {elapsed_time:.2f} seconds')
    plt.xlabel('Line Number')
    plt.ylabel('Sentiment')
    plt.title(f'Sentiments Over Time - Model: {model_name} - File: {file_name} ')
    plt.legend()
    plt.show()
    return table_Sent

#perform the analyse
file_name = "med.txt"

#Datentabelle für Gen
table_Sent_gen = pd.DataFrame(columns=['idx','Sprecher', 'Text', openai_config_3['model_name'],openai_config_4['model_name'], openai_config_13B['model_name'], openai_config_70B['model_name'],openai_config_pol['model_name']])



print("gen GPT3.5")
table_Sent_gen = analyze_sentiments(openai_config_3, lines_gen, file_name, table_Sent_gen)
print("gen GPT4")
table_Sent_gen = analyze_sentiments(openai_config_4, lines_gen, file_name, table_Sent_gen)
print("gen 13B")
table_Sent_gen = analyze_sentiments(openai_config_13B, lines_gen, file_name, table_Sent_gen)

print("gen 70B")
table_Sent_gen =  analyze_sentiments(openai_config_70B, lines_gen, file_name, table_Sent_gen)
print("gen pol")
table_Sent_gen = analyze_sentiments(openai_config_pol, lines_gen, file_name, table_Sent_gen)

print(table_Sent_gen) 
table_Sent_gen.to_excel("sentimen_gen.xlsx", index=False)


#realBsp --> only local LLM 13B and 70B 

table_Sent_real = pd.DataFrame(columns=['idx','Sprecher', 'Text', openai_config_13B['model_name'], openai_config_70B['model_name'],openai_config_pol['model_name']])

file_name = "realBsp.txt"
print("real 13B")
table_Sent_real = analyze_sentiments(openai_config_13B, lines_real, file_name ,table_Sent_real)
print("real 70B")
table_Sent_real = analyze_sentiments(openai_config_70B, lines_real, file_name,table_Sent_real)
print("real pol")
table_Sent_real = analyze_sentiments(openai_config_pol, lines_real, file_name,table_Sent_real)
print(table_Sent_real)
table_Sent_real.to_excel("sentimen_real.xlsx", index=False)

Kontextbasierte Fragen an das LLM

In [ ]:
import time
from textblob import TextBlob
import pandas as pd

# Spezifische Fragen
questions = [
    "Which mediation technique is used in the following conversation?",
    "What was the spirit like during the mediation? ",
    "What were the main reasons for the conflict in the mediationtranscription? ",
    "How much of the conversation does each participant have?",
    "Is the mediation technique appropriate, or is there room for improvement?"
]

temperature=0.0


# Send the user prompt to ChatGPT using the chat/completions endpoint
def ask_question_to_gpt(prompt,model_name):
     
    response = openai.ChatCompletion.create(
    model=model_name,      
    messages=[
        {"role": "system", "content": "You are a helpful assistant and give short correct answers"},
        {"role": "user", "content": prompt}
    ],
    #max_tokens=-1,
    temperature = temperature,
    timeout=1000,
    top_k=2
    )
    return response['choices'][0]['message']['content'].strip()



def ask_questions(openai_config, conversation_text, table_Quest):
    if 'model_name' in openai_config:
        openai.model_name = openai_config['model_name']
        
    if 'key' in openai_config:
        openai.api_key = openai_config['key']
    if 'base' in openai_config:
        openai.api_base = openai_config['base']
  
    for question in questions:
        answer = ask_question_to_gpt(f"Frage: {question} zur Mediation: [{conversation_text}]",openai_config['model_name'])
        if not any(table_Quest['Frage'] == question):
         # Füge eine neue Zeile für  hinzu, falls sie nicht existiert
                table_Quest = table_Quest.append({'Frage': question }, ignore_index=True)

        # Wert für eine bestimmte Zelle setzen, auch wenn die Zeile gerade erst hinzugefügt wurde
        table_Quest.at[table_Quest[table_Quest['Frage'] == question].index[0], openai_config['model_name']] = answer
        print(f"Frage: {question}\nAntwort: {answer}\n")


    # Drucken des Models und der Quelle
    print(f"Model: {openai_config['model_name']} - \n")
    return table_Quest


#perform the analyse

file_name = "med.txt"
table_Quest_gen = pd.DataFrame(columns=['Frage', openai_config_3['model_name'],openai_config_4['model_name'], openai_config_13B['model_name'], openai_config_70B['model_name']])

print("gen GPT3.5")
#table_Quest_gen = ask_questions(openai_config_3, conversation_text_gen, table_Quest_gen)
print("gen GPT4")
#table_Quest_gen = ask_questions(openai_config_4, conversation_text_gen, table_Quest_gen)
print("gen 13B")
#table_Quest_gen = ask_questions(openai_config_13B, conversation_text_gen, table_Quest_gen)
print("gen 70B")
table_Quest_gen = ask_questions(openai_config_70B, conversation_text_gen, table_Quest_gen)
print(table_Quest_gen)

#table_Quest_gen.to_excel("questian_gen.xlsx", index=False)

file_name = "realBsp.txt"
#realBsp --> only local LLM 13B and 70B 
table_Quest_real = pd.DataFrame(columns=['Frage', openai_config_13B['model_name'], openai_config_70B['model_name']])

print("real 13B")
#table_Quest_real = ask_questions(openai_config_13B, conversation_text_real, table_Quest_real)
print("real 70B")
table_Quest_real = ask_questions(openai_config_70B, conversation_text_real, table_Quest_real)

table_Quest_real.to_excel("questian_real.xlsx", index=False)

Ausgabe der Datentabellen 

In [ ]:
print('\nDatentabelle Sentiment für generierte Mediation\n')
print(table_Sent_gen)

print('\nDatentabelle Sentiment für reale Mediation\n')
print(table_Sent_real)

print('\nDatentabelle Fragen für generierte Mediation\n')
print(table_Quest_gen)

print('\nDatentabelle Fragen für reale Mediation\n')
print(table_Quest_real)